In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Import the necessary library
from google.colab import drive

# Step 2: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load datasets
train_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/train_with_variogram.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/test_with_variogram.csv')
train_gap_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Gap_Train.csv')
test_gap_df = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/Gap_Test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/SampleSubmission.csv')

In [4]:
train_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,xhp20_sill,xhp20_range,xhp20_nugget,xhp20_spatial_dependency,xhp20_variogram_avg,BulkDensity_sill,BulkDensity_range,BulkDensity_nugget,BulkDensity_spatial_dependency,BulkDensity_variogram_avg
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,1.061376,2.72674,0.176896,0.833333,0.041199,0.864881,2.72674,0.288294,0.666667,1.228435
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,1.061376,2.72674,0.176896,0.833333,0.041023,0.864881,2.72674,0.288294,0.666667,1.228432
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,1.061376,2.72674,0.176896,0.833333,0.040979,0.864881,2.72674,0.288294,0.666667,1.228414
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,1.061376,2.72674,0.176896,0.833333,0.040947,0.864881,2.72674,0.288294,0.666667,1.228352
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,1.061376,2.72674,0.176896,0.833333,0.041268,0.864881,2.72674,0.288294,0.666667,1.228384


In [5]:
test_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,xhp20_sill,xhp20_range,xhp20_nugget,xhp20_spatial_dependency,xhp20_variogram_avg,BulkDensity_sill,BulkDensity_range,BulkDensity_nugget,BulkDensity_spatial_dependency,BulkDensity_variogram_avg
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,...,1.061376,2.72674,0.176896,0.833333,0.041636,0.864881,2.72674,0.288294,0.666667,1.218624
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,...,1.061376,2.72674,0.176896,0.833333,0.042926,0.864881,2.72674,0.288294,0.666667,1.218698
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,...,1.061376,2.72674,0.176896,0.833333,0.042560,0.864881,2.72674,0.288294,0.666667,1.218678
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,...,1.061376,2.72674,0.176896,0.833333,0.042172,0.864881,2.72674,0.288294,0.666667,1.218775
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,...,1.061376,2.72674,0.176896,0.833333,0.042397,0.864881,2.72674,0.288294,0.666667,1.218449


In [6]:
train_gap_df.head()

,Nutrient,Required,Available,Gap,PID
0,N,100.0,3796.0000,-3696.0000,ID_I5RGjv
1,P,40.0,0.9928,39.0072,ID_I5RGjv
2,K,52.0,429.2400,-377.2400,ID_I5RGjv
3,Ca,12.0,19943.6000,-19931.6000,ID_I5RGjv
4,Mg,8.0,6745.2000,-6737.2000,ID_I5RGjv


In [7]:
test_gap_df = pd.merge(test_gap_df, test_df[['PID', 'BulkDensity']], on='PID', how='left')

In [8]:
test_gap_df.head()

,Nutrient,Required,PID,BulkDensity
0,N,100.0,ID_NGS9Bx,1.2
1,P,40.0,ID_NGS9Bx,1.2
2,K,52.0,ID_NGS9Bx,1.2
3,Ca,12.0,ID_NGS9Bx,1.2
4,Mg,8.0,ID_NGS9Bx,1.2


In [9]:
sample_submission.head()

,ID,Gap
0,ID_002W8m_B,0
1,ID_002W8m_Ca,0
2,ID_002W8m_Cu,0
3,ID_002W8m_Fe,0
4,ID_002W8m_K,0


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

# Define target columns and use your feature sets as before
target_columns = ['Cu', 'Mg', 'S', 'Mn', 'Zn', 'Fe', 'B', 'N', 'P', 'K', 'Ca']

feature_sets = {
    'N': ['ph20', 'mb1_variogram_avg', 'mb2_variogram_avg', 'mdem', 'pH', 'mb3_variogram_avg',
          'ls_variogram_avg', 'cec20_variogram_avg', 'parv_variogram_avg', 'slope', 'bio7', 'mb7_variogram_avg',
          'mb1', 'ph20_variogram_avg', 'soc20', 'lstn_variogram_avg'],
    'P': ['ph20', 'ph20_variogram_avg', 'BulkDensity_variogram_avg'],
    'Mg': ['pH', 'snd20_variogram_avg', 'mb2_variogram_avg', 'mb7', 'pH_variogram_avg', 'mdem', 'mb3_variogram_avg', 'ph20_variogram_avg', 'soc20_variogram_avg', 'slope', 'ls_variogram_avg', 'dows'],
    'Ca': ['pH', 'bio7_variogram_avg', 'mb1_variogram_avg', 'mb7', 'ecec20_variogram_avg', 'lstd_variogram_avg',
           'mb3_variogram_avg', 'mb2', 'BulkDensity_variogram_avg', 'slope', 'cec20_variogram_avg', 'ph20_variogram_avg'],
    'S': ['pH', 'soc20_variogram_avg', 'ph20_variogram_avg', 'BulkDensity_variogram_avg', 'dows', 'parv_variogram_avg'],
    'K': ['pH', 'ls_variogram_avg', 'pH_variogram_avg', 'bio7_variogram_avg', 'mdem', 'BulkDensity', 'bio7',
          'BulkDensity_variogram_avg', 'alb', 'bp', 'bio12', 'cec20_variogram_avg', 'mb3'],
    'Mn': ['pH', 'mb1_variogram_avg', 'lstd_variogram_avg', 'mb2_variogram_avg', 'lstn', 'slope_variogram_avg', 'lstn_variogram_avg', 'mdem', 'alb', 'mb3_variogram_avg', 'mdem_variogram_avg',
           'snd20_variogram_avg', 'bio7_variogram_avg', 'ph20_variogram_avg', 'bio12', 'alb_variogram_avg', 'pH_variogram_avg', 'bio1_variogram_avg'],
    'Zn': ['bio7', 'pH_variogram_avg', 'pH', 'mdem_variogram_avg', 'alb', 'mb3_variogram_avg'],
    'Fe': ['pH', 'alb_variogram_avg', 'snd20_variogram_avg', 'ph20_variogram_avg', 'dows_variogram_avg', 'slope_variogram_avg', 'mb2_variogram_avg', 'bio7', 'ph20', 'alb', 'bio1_variogram_avg', 'cec20_variogram_avg', 'hp20_variogram_avg', 'dows'],
    'B': ['pH', 'alb_variogram_avg', 'BulkDensity_variogram_avg', 'lstd_variogram_avg',
          'mdem', 'ls_variogram_avg', 'bio7_variogram_avg', 'mb3_variogram_avg', 'ph20', 'alb', 'mdem_variogram_avg'],
    'Cu': ['bio7', 'mb3_variogram_avg', 'alb_variogram_avg', 'mb7', 'parv_variogram_avg', 'lstd', 'pH', 'mdem_variogram_avg', 'snd20_variogram_avg']

}

# Best hyperparameters for RandomForest models
rf_best_params = {
    'Cu': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'Mg': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'S': {'n_estimators': 800, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True},
    'Mn': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'Zn': {'n_estimators': 200, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_depth': 12},
    'Fe': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'B': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'N': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'P': {'n_estimators': 800, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': None, 'bootstrap': False},
    'K': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False},
    'Ca': {'n_estimators': 800, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': False}
}

# Best hyperparameters for ExtraTrees models
et_best_params = {
    'Cu': {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'Mg': {'n_estimators': 100, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 30, 'bootstrap': False},
    'S': {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': None, 'max_depth': 10, 'bootstrap': True},
    'Mn': {'n_estimators': 100, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 30, 'bootstrap': False},
    'Zn': {'n_estimators': 1000, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'Fe': {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'B': {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'N': {'n_estimators': 100, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 30, 'bootstrap': False},
    'P': {'n_estimators': 1000, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'K': {'n_estimators': 1000, 'min_samples_split': 8, 'min_samples_leaf': 4, 'max_features': None, 'max_depth': 20, 'bootstrap': False},
    'Ca': {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 20, 'bootstrap': False}
}

# Function to find optimal ensemble weights
def find_optimal_weights(y_true, rf_pred, et_pred):
    """
    Find optimal weights for ensemble using scipy's minimize function
    """
    def mae_loss(weights):
        # Ensure weights sum to 1
        weights = np.array(weights)
        weights = weights / np.sum(weights)

        # Create weighted ensemble prediction
        ensemble_pred = weights[0] * rf_pred + weights[1] * et_pred

        # Return MAE (to minimize)
        return mean_absolute_error(y_true, ensemble_pred)

    # Initial weights (equal)
    initial_weights = [0.5, 0.5]

    # Constraint: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

    # Bounds: weights between 0 and 1
    bounds = [(0, 1), (0, 1)]

    # Find optimal weights
    result = minimize(mae_loss, initial_weights, method='SLSQP',
                     bounds=bounds, constraints=constraints)

    # Normalize weights to ensure they sum to 1
    optimal_weights = result['x'] / np.sum(result['x'])

    return optimal_weights

# Function to create and evaluate optimized ensemble for a target
def build_optimized_ensemble(X_train, X_test, y_train, y_test, target):
    # Initialize models with best parameters
    rf = RandomForestRegressor(random_state=42, **rf_best_params[target])
    et = ExtraTreesRegressor(random_state=42, **et_best_params[target])

    # Fit models
    rf.fit(X_train, y_train)
    et.fit(X_train, y_train)

    # Make predictions
    rf_pred = rf.predict(X_test)
    et_pred = et.predict(X_test)

    # Calculate individual model MAEs
    rf_mae = mean_absolute_error(y_test, rf_pred)
    et_mae = mean_absolute_error(y_test, et_pred)

    # Find optimal weights
    optimal_weights = find_optimal_weights(y_test, rf_pred, et_pred)

    # Create ensemble prediction with optimal weights
    ensemble_pred = optimal_weights[0] * rf_pred + optimal_weights[1] * et_pred
    ensemble_mae = mean_absolute_error(y_test, ensemble_pred)

    return {
        'RandomForest': rf_mae,
        'ExtraTrees': et_mae,
        'Optimized Ensemble': ensemble_mae,
        'RF Weight': optimal_weights[0],
        'ET Weight': optimal_weights[1],
        'RF Model': rf,
        'ET Model': et
    }

# Results dictionary
results = {}

# Process each target
def train_models(train_df):
    for target in target_columns:
        print(f"\nProcessing target: {target}")
        features = feature_sets.get(target, [])

        if not features:
            print(f"  No features defined for {target}, skipping...")
            continue

        # Prepare data
        X = train_df[features].fillna(0)
        y = train_df[target]

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Build and evaluate optimized ensemble
        target_results = build_optimized_ensemble(
            X_train_scaled, X_test_scaled, y_train, y_test, target
        )

        results[target] = target_results

        print(f"  RandomForest MAE: {target_results['RandomForest']:.4f}")
        print(f"  ExtraTrees MAE: {target_results['ExtraTrees']:.4f}")
        print(f"  Optimized Ensemble MAE: {target_results['Optimized Ensemble']:.4f}")
        print(f"  Optimal Weights - RF: {target_results['RF Weight']:.4f}, ET: {target_results['ET Weight']:.4f}")

    # Create summary table
    summary_data = {
        'Target': [],
        'RandomForest': [],
        'ExtraTrees': [],
        'Optimized Ensemble': [],
        'RF Weight': [],
        'ET Weight': [],
        'Improvement (%)': []
    }

    for target, res in results.items():
        # Calculate improvement over best individual model
        best_individual = min(res['RandomForest'], res['ExtraTrees'])
        improvement = (best_individual - res['Optimized Ensemble']) / best_individual * 100

        summary_data['Target'].append(target)
        summary_data['RandomForest'].append(res['RandomForest'])
        summary_data['ExtraTrees'].append(res['ExtraTrees'])
        summary_data['Optimized Ensemble'].append(res['Optimized Ensemble'])
        summary_data['RF Weight'].append(res['RF Weight'])
        summary_data['ET Weight'].append(res['ET Weight'])
        summary_data['Improvement (%)'].append(improvement)

    # Create summary DataFrame
    summary_df = pd.DataFrame(summary_data)

    # Print summary table
    print("\n" + "="*80)
    print("OPTIMIZED ENSEMBLE RESULTS SUMMARY")
    print("="*80)
    print(summary_df.to_string(index=False, float_format=lambda x: f"{x:.4f}"))

    return summary_df

# Save the best models for each target
def save_best_models():
    best_models = {}
    for target, res in results.items():
        if res['Optimized Ensemble'] < min(res['RandomForest'], res['ExtraTrees']):
            # Save optimized ensemble info
            best_models[target] = {
                'type': 'Optimized Ensemble',
                'RF': res['RF Model'],
                'ET': res['ET Model'],
                'RF Weight': res['RF Weight'],
                'ET Weight': res['ET Weight']
            }
        elif res['RandomForest'] <= res['ExtraTrees']:
            # Save RandomForest
            best_models[target] = {
                'type': 'RandomForest',
                'model': res['RF Model']
            }
        else:
            # Save ExtraTrees
            best_models[target] = {
                'type': 'ExtraTrees',
                'model': res['ET Model']
            }
    return best_models

# Define a prediction function for the test set
def predict_with_best_models(test_df, train_df, best_models):
    """
    Make predictions on test data using the best models for each target

    Args:
        test_df: The test dataframe with all features
        train_df: The training dataframe used to fit the scaler
        best_models: Dictionary of best models for each target

    Returns:
        DataFrame with predictions for each target and PID column
    """
    predictions = {}

    # First, extract the PID column
    if 'PID' in test_df.columns:
        predictions['PID'] = test_df['PID'].values
    else:
        # If PID column doesn't exist, create a default index
        predictions['PID'] = np.arange(len(test_df))
        print("Warning: 'PID' column not found in test_df. Using index as PID.")

    for target in target_columns:
        features = feature_sets.get(target, [])
        if not features or target not in best_models:
            continue

        # Extract features for this target
        X_test = test_df[features].fillna(0)

        # Scale features
        scaler = StandardScaler()
        X_train = train_df[features].fillna(0)
        scaler.fit(X_train)  # Fit on training data
        X_test_scaled = scaler.transform(X_test)

        # Get model info
        model_info = best_models[target]

        # Make prediction based on model type
        if model_info['type'] == 'Optimized Ensemble':
            # Get predictions from both models
            rf_pred = model_info['RF'].predict(X_test_scaled)
            et_pred = model_info['ET'].predict(X_test_scaled)

            # Combine with optimal weights
            pred = model_info['RF Weight'] * rf_pred + model_info['ET Weight'] * et_pred
        else:
            # Use single model
            pred = model_info['model'].predict(X_test_scaled)

        predictions[target] = pred

    # Combine all predictions into a DataFrame
    df_predictions = pd.DataFrame(predictions)

    # Ensure PID is the first column
    cols = df_predictions.columns.tolist()
    cols.insert(0, cols.pop(cols.index('PID')))
    df_predictions = df_predictions[cols]

    return df_predictions


# Train models
summary_df = train_models(train_df)
# Save best models
best_models = save_best_models()
# Make predictions on test data
test_predictions = predict_with_best_models(test_df, train_df, best_models)


Processing target: Cu
  RandomForest MAE: 0.8841
  ExtraTrees MAE: 0.8889
  Optimized Ensemble MAE: 0.8821
  Optimal Weights - RF: 0.5896, ET: 0.4104

Processing target: Mg
  RandomForest MAE: 209.5482
  ExtraTrees MAE: 212.6613
  Optimized Ensemble MAE: 209.3958
  Optimal Weights - RF: 0.8529, ET: 0.1471

Processing target: S
  RandomForest MAE: 3.1630
  ExtraTrees MAE: 3.2920
  Optimized Ensemble MAE: 3.1630
  Optimal Weights - RF: 1.0000, ET: 0.0000

Processing target: Mn
  RandomForest MAE: 34.0286
  ExtraTrees MAE: 33.8503
  Optimized Ensemble MAE: 33.7461
  Optimal Weights - RF: 0.3823, ET: 0.6177

Processing target: Zn
  RandomForest MAE: 0.8904
  ExtraTrees MAE: 0.8832
  Optimized Ensemble MAE: 0.8803
  Optimal Weights - RF: 0.3737, ET: 0.6263

Processing target: Fe
  RandomForest MAE: 28.3343
  ExtraTrees MAE: 28.3911
  Optimized Ensemble MAE: 28.1962
  Optimal Weights - RF: 0.5149, ET: 0.4851

Processing target: B
  RandomForest MAE: 0.1301
  ExtraTrees MAE: 0.1329
  Optimiz

In [13]:
test_predictions.head()

,PID,Cu,Mg,S,Mn,Zn,Fe,B,N,P,K,Ca
0,ID_NGS9Bx,3.584856,1339.114656,7.781161,118.999922,1.401206,108.469816,0.269287,1306.480339,5.772913,153.286186,5179.032289
1,ID_YdVKXw,4.135801,1613.327748,7.420201,144.668437,1.441999,117.972125,0.357978,1375.073875,6.672270,178.769664,6087.535608
2,ID_MZAlfE,4.055788,1292.343879,8.099532,131.696565,1.769157,123.028650,0.274678,1658.455757,7.623112,173.808453,4644.552499
3,ID_GwCCMN,4.085820,1462.431114,7.671036,128.660272,1.589176,114.835067,0.273868,1313.210778,6.010307,170.675369,5331.332204
4,ID_K8sowf,4.339874,1319.800607,8.110701,128.632835,1.885846,120.300336,0.266667,1553.963571,13.124763,148.192911,3972.018830


In [14]:
submission = test_predictions

In [15]:
submission.head()

,PID,Cu,Mg,S,Mn,Zn,Fe,B,N,P,K,Ca
0,ID_NGS9Bx,3.584856,1339.114656,7.781161,118.999922,1.401206,108.469816,0.269287,1306.480339,5.772913,153.286186,5179.032289
1,ID_YdVKXw,4.135801,1613.327748,7.420201,144.668437,1.441999,117.972125,0.357978,1375.073875,6.672270,178.769664,6087.535608
2,ID_MZAlfE,4.055788,1292.343879,8.099532,131.696565,1.769157,123.028650,0.274678,1658.455757,7.623112,173.808453,4644.552499
3,ID_GwCCMN,4.085820,1462.431114,7.671036,128.660272,1.589176,114.835067,0.273868,1313.210778,6.010307,170.675369,5331.332204
4,ID_K8sowf,4.339874,1319.800607,8.110701,128.632835,1.885846,120.300336,0.266667,1553.963571,13.124763,148.192911,3972.018830


In [16]:
# prompt: turn submission into a 3 column file that has the column PID, Nutrient, Value
submission_melted = submission.melt(id_vars=['PID'], var_name='Nutrient', value_name='Available_Nutrients_in_ppm')
submission_melted = submission_melted.sort_values('PID')
submission_melted.head()

,PID,Nutrient,Available_Nutrients_in_ppm
19869,ID_002W8m,P,6.239099
15033,ID_002W8m,B,0.375078
2943,ID_002W8m,Mg,561.158299
24705,ID_002W8m,Ca,2692.421186
525,ID_002W8m,Cu,3.376701


In [17]:
# prompt: merge test_gap_df with submission_melted on PID and Nutrient
nutrient_df = pd.merge(test_gap_df, submission_melted, on=['PID', 'Nutrient'], how='left')

In [18]:
soil_depth = 20  # cm

# Calculate the Available_Nutrients_in_kg_ha
nutrient_df['Available_Nutrients_in_kg_ha'] = (nutrient_df['Available_Nutrients_in_ppm']
                                               * soil_depth * nutrient_df['BulkDensity'] * 0.1)

In [19]:
nutrient_df.head()

,Nutrient,Required,PID,BulkDensity,Available_Nutrients_in_ppm,Available_Nutrients_in_kg_ha
0,N,100.0,ID_NGS9Bx,1.2,1306.480339,3135.552813
1,P,40.0,ID_NGS9Bx,1.2,5.772913,13.854990
2,K,52.0,ID_NGS9Bx,1.2,153.286186,367.886845
3,Ca,12.0,ID_NGS9Bx,1.2,5179.032289,12429.677493
4,Mg,8.0,ID_NGS9Bx,1.2,1339.114656,3213.875173


In [20]:
nutrient_df["Gap"] = nutrient_df["Required"] - nutrient_df["Available_Nutrients_in_kg_ha"]

In [21]:
nutrient_df['ID'] = nutrient_df['PID'] + "_" + nutrient_df['Nutrient']
nutrient_df = nutrient_df[['ID', 'Gap']]
nutrient_df.head()

,ID,Gap
0,ID_NGS9Bx_N,-3035.552813
1,ID_NGS9Bx_P,26.145010
2,ID_NGS9Bx_K,-315.886845
3,ID_NGS9Bx_Ca,-12417.677493
4,ID_NGS9Bx_Mg,-3205.875173


In [22]:
nutrient_df.to_csv('/content/drive/MyDrive/Amini Soil Prediction Challenge/experiment.csv', index=False)
print("Submission file saved asexperiment.csv")

Submission file saved asexperiment.csv
